In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Codingan Fine-Tuning Skripsiiiii

In [6]:
#pip install transformers[torch]

In [7]:
#pip install jiwer

In [8]:
#pip install datasets

## Library

In [ ]:
import random
import pandas as pd
import re
from datasets import load_dataset, load_metric
from IPython.display import display, HTML
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor
import json
import torchaudio
import numpy as np
import IPython.display as ipd

## Dataset

In [6]:
dataset = load_dataset("jaal047/profanity-speech-suroboyoan")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Resolving data files:   0%|          | 0/250 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/50 [00:00<?, ?it/s]

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 250
    })
    test: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 50
    })
})

## Prepare Data, Tokenizer, Feature Extractor

In [8]:
data_train = dataset['train']
data_test = dataset['test']

In [9]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [10]:
show_random_elements(data_train.remove_columns(["audio"]), num_examples=20)

,transcription
0,Hayo koen lapo iku ancene arek kelakuan koyok asu iku mobil goblok guduk kanvas iki wes koen coreti opo ae jangkrik ndang hapus tak sikat silit mu lek gak resik
1,We cok alon alon lek nyetir dalan mudun iki goblok ojok malah digas ngentot lek tiboh soro kene
2,Arek iki ancen bedo cok ndue aib malah dibangga banggakno ancen goblok longor tolol wes gak isok berkata kata aku karo kelakuan mu seng koyok taek iki
3,Cok karyawan koyok ngene iki lo seng nggateli jangkrek gak kerjo plonga plongo tok ae asu asu wes tala abot
4,Yo gak ngono cara maine nggateli iki lek koyok ngono yo awakmu ga isok kalah goblok
5,Lah koen ndek kene lapo ae plonga plongo koyok wong goblok inisiatif ngunu loh takon takon ta lapo malah meneng nang pojokan koyok taek
6,Arek iki ancen bedo cok ndue aib malah dibangga banggakno ancen goblok longor tolol wes gak isok berkata kata aku karo kelakuan mu seng koyok taek iki
7,Mesiyo wingi wes tak bantu sampe saiki arek e jek tetep ga iso ancen tolol arek iku longor wis ga ketolong
8,Ealah mek tak seleh sedilut ae raine wes koyok silit ngene sorry a maeng si anjing iku nggupuhi aku dadi langsung tak seleh ga ngomong ngomong
9,Jancok mogok maneh cok cok montor gatel padahal wingi wes tak gowo nak bengkel tapi kok jek kambuh maneh iki asu asu


In [11]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\'\”_\�]'
#chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["transcription"] = re.sub(chars_to_ignore_regex, '', batch["transcription"]).lower() + " "
    batch["transcription"] = batch["transcription"].replace('！ ', '')
    batch["transcription"] = batch["transcription"].replace('，', '')
    batch["transcription"] = batch["transcription"].replace('é', 'e')
    batch["transcription"] = batch["transcription"].replace('è', 'e')
    return batch

In [12]:
data_train = data_train.map(remove_special_characters)
data_test = data_test.map(remove_special_characters)

In [13]:
show_random_elements(data_train.remove_columns(["audio"]))

,transcription
0,ealah mek tak seleh sedilut ae raine wes koyok silit ngene sorry a maeng si anjing iku nggupuhi aku dadi langsung tak seleh ga ngomong ngomong
1,peda mu ojok parkir kono cok ngalangi dalan goblok utek mu nak endi ae se ajane nak silit ta ojok longor longor po o lek dadi arek
2,lek koen dodol opo ae se iki gajelas panganan rupane koyok taek ombenan ambune amis cok bakul taek dodol gatel a koen iku pantes ae gak onok seng gelem tuku
3,halah taek raimu omangan mu ga isok dicekel asu wingi ae jare arep mbalikno kelambiku sampe saiki jek gurung dibalekno bangsat
4,hancok kegolan maneh taek taek nggatel aku dolen cek ga mumet malah kalah bal balan asu kene koen jancok mbok kiro aku arek sepele
5,jancok arek iki koen ga ngerti wong jek bingung malah digupui goblok
6,ih cok kemproh kaos kaki teles kat maeng isuk jek mbok gae ae keon mikir opo seh ajane goblok ambune iku loh cok koyok jembut ndang kumbahen kono lo asu
7,halo koen nangndi saiki bangsat aku wes ngenteni nak kene sak jaman ngentot koyok wong goblok aku ngenteni awakmu nak kene ndang rene asu
8,wes ga kuat aku cok ngerjakno soal iki kongkonen bajingan seng turu iku ae gak lapo lapo kat maeng bangsat
9,mesiyo wingi wes tak bantu sampe saiki arek e jek tetep ga iso ancen tolol arek iku longor wis ga ketolong


In [14]:
def extract_all_chars(batch):
  all_text = " ".join(batch["transcription"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [15]:
vocab_train = data_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=data_train.column_names)
vocab_test = data_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=data_test.column_names)

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [16]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [17]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'o': 0,
 'r': 1,
 'h': 2,
 'd': 3,
 ' ': 4,
 'n': 5,
 'p': 6,
 'a': 7,
 'u': 8,
 'b': 9,
 'v': 10,
 'y': 11,
 'e': 12,
 'c': 13,
 's': 14,
 'k': 15,
 't': 16,
 'j': 17,
 'm': 18,
 'w': 19,
 'i': 20,
 'f': 21,
 'g': 22,
 'l': 23}

In [18]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [19]:
vocab_dict[""] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

26

In [20]:
import json
with open('vocab-jv.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [21]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("vocab-jv.json", unk_token="", pad_token="[PAD]", word_delimiter_token="|")

In [22]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [23]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [24]:
processor.save_pretrained("/content/drive/MyDrive/Skripsi/wav2vec2-profanity-javanese-sby")

[]

## Data Preprocessing

In [25]:
data_train[14]

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/0713df7cbe35949f7abe7faf32e5468da54305ad5d1f354b1d443df56af3e58a',
  'array': array([ 0.00000000e+00,  0.00000000e+00, -3.05175781e-05, ...,
          0.00000000e+00,  0.00000000e+00, -3.05175781e-05]),
  'sampling_rate': 16000},
 'transcription': 'lo guduk ngunu cok carane goblok iki sekta tak jelasno sek baru garapen bangsat '}

In [26]:
import torchaudio

def load_audio(batch):
    audio_array, sampling_rate = torchaudio.load(batch["audio"]["path"])
    batch["speech"] = audio_array[0].numpy()  # Memuat audio ke dalam bentuk numpy array
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["transcription"]
    return batch

In [27]:
data_train = data_train.map(load_audio, remove_columns=data_train.column_names)
data_test = data_test.map(load_audio, remove_columns=data_test.column_names)

In [28]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(data_train))

ipd.Audio(data=np.asarray(data_train[rand_int]["speech"]), autoplay=True, rate=16000)

In [29]:
rand_int = random.randint(0, len(data_test))
ipd.Audio(data=np.asarray(data_test[rand_int]["speech"]), autoplay=True, rate=16000)

In [30]:
rand_int = random.randint(0, len(data_train))

print("Target text:", data_train[rand_int]["target_text"])
print("Input array shape:", np.asarray(data_train[rand_int]["speech"]).shape)
print("Sampling rate:", data_train[rand_int]["sampling_rate"])

Target text: oi cok koen tekan endi ae ga tau ketok raimu nak njobo kene sek ae lo ngobrol ngobrol sek karo aku 
Input array shape: (99328,)
Sampling rate: 16000


In [31]:
# Fungsi untuk mempersiapkan dataset
def prepare_dataset(batch):
    # Check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values

    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [32]:
# Terapkan prepare_dataset ke data_train dan data_test
data_train = data_train.map(prepare_dataset, remove_columns=data_train.column_names, batch_size=8, num_proc=4, batched=True)
data_test = data_test.map(prepare_dataset, remove_columns=data_test.column_names, batch_size=8, num_proc=4, batched=True)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/250 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call

Map (num_proc=4):   0%|          | 0/50 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call

In [33]:
#Cek konversi sudah benar atau belum

decoded_text = tokenizer.decode(data_train["labels"][14], skip_special_tokens=True)
print("Teks semula:", decoded_text)

Teks semula: lo guduk ngunu cok carane goblok iki sekta tak jelasno sek baru garapen bangsat


## SetUp Trainer

In [34]:
import torch
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC, TrainingArguments, Trainer

processor = Wav2Vec2Processor.from_pretrained("indonesian-nlp/wav2vec2-indonesian-javanese-sundanese")

@dataclass
class DataCollatorCTCWithPadding:
    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [35]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [36]:
# Periksa panjang setiap fitur
#for feature in data_train:
    #input_length = len(feature["input_values"])
    #label_length = len(feature["labels"])
    #print(f"Input length: {input_length}, Label length: {label_length}")

In [37]:
wer_metric = load_metric("wer")

<ipython-input-37-605de035dfd7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for wer contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/wer/wer.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [38]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [39]:
model = Wav2Vec2ForCTC.from_pretrained(
    "indonesian-nlp/wav2vec2-indonesian-javanese-sundanese",
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.3,
    layerdrop=0.1,
    gradient_checkpointing=True,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
    ctc_zero_infinity=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at indonesian-nlp/wav2vec2-indonesian-javanese-sundanese were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights

In [40]:
model.freeze_feature_extractor()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1914: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [41]:
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Skripsi/wav2vec2-profanity-javanese-sby",
    group_by_length=True,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    evaluation_strategy="steps",
    num_train_epochs=80,
    fp16=True,
    save_steps=200,
    eval_steps=200,
    logging_steps=200,
    learning_rate=1e-4,
    warmup_steps=500,
    save_total_limit=2,
)

In [42]:
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=data_train,
    eval_dataset=data_test,
    tokenizer=processor.feature_extractor,
)

In [43]:
torch.cuda.is_available()

True

In [44]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
200,3.196300,0.438085,0.536998
400,0.592900,0.129732,0.232558
600,0.340100,0.076427,0.150106
800,0.235700,0.059960,0.112051
1000,0.186500,0.055625,0.099366
1200,0.166500,0.051896,0.088795


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call

TrainOutput(global_step=1200, training_loss=0.7863523006439209, metrics={'train_runtime': 4539.8167, 'train_samples_per_second': 4.405, 'train_steps_per_second': 0.264, 'total_flos': 5.022634798903542e+18, 'train_loss': 0.7863523006439209, 'epoch': 76.19047619047619})

In [45]:
trainer.save_model()
trainer.save_state()

In [46]:
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torchaudio

# Load model dan processor
model_path = "/content/drive/MyDrive/Skripsi/wav2vec2-profanity-javanese-sby"
#model_path = "indonesian-nlp/wav2vec2-indonesian-javanese-sundanese"
model = Wav2Vec2ForCTC.from_pretrained(model_path)
processor = Wav2Vec2Processor.from_pretrained(model_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [47]:
# Load vocab untuk memeriksa isinya
with open(f"{model_path}/vocab.json", 'r') as f:
    vocab = json.load(f)

print("Vocab:", vocab)

Vocab: {'': 24, '[PAD]': 25, 'a': 7, 'b': 9, 'c': 13, 'd': 3, 'e': 12, 'f': 21, 'g': 22, 'h': 2, 'i': 20, 'j': 17, 'k': 15, 'l': 23, 'm': 18, 'n': 5, 'o': 0, 'p': 6, 'r': 1, 's': 14, 't': 16, 'u': 8, 'v': 10, 'w': 19, 'y': 11, '|': 4}


In [48]:
# Fungsi untuk memuat dan preprocess audio
def load_and_preprocess_audio(file_path):
    audio_array, sampling_rate = torchaudio.load(file_path)
    if sampling_rate != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=sampling_rate, new_freq=16000)
        audio_array = resampler(audio_array)
    audio_array = audio_array.squeeze().numpy()
    return audio_array

# Path ke file audio
audio_path = "./audio_165.wav"

In [49]:
# Muat dan preprocess audio
audio_array = load_and_preprocess_audio(audio_path)

# Preprocess audio dengan processor
inputs = processor(audio_array, sampling_rate=16000, return_tensors="pt", padding=True)

# Model inferensi
with torch.no_grad():
    logits = model(inputs.input_values).logits

# Ambil argmax dari logits untuk mendapatkan prediksi ID
predicted_ids = torch.argmax(logits, dim=-1)

In [50]:
predicted_ids

tensor([[29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
         29, 29, 29, 29, 29, 29, 17, 29, 29,  7, 29, 29, 29, 29, 29,  5,  5, 29,
         29, 29, 13, 29,  0, 29, 29, 15,  4,  4,  7, 29,  1,  1, 12, 15, 15,  4,
          4, 20, 20, 15, 29, 20, 29,  4,  4, 29, 29, 29, 29, 29, 29, 29, 29, 15,
          0, 12, 12,  5,  5,  4,  4, 29, 29, 22,  7, 29,  4,  4,  5,  5, 22, 29,
         12, 29,  1,  1, 29, 29, 29, 16, 29, 20, 29, 29, 29, 29,  7, 29, 29,  4,
          4, 19,  0,  0, 29,  5, 22,  4,  4, 29, 17, 29, 12, 15, 15,  4,  4, 29,
         29,  9, 29, 20, 29, 29,  5, 22, 22,  8, 29,  5,  5, 22, 22,  4,  4, 29,
         29, 29, 29, 29, 29, 29, 29, 29, 18, 29,  7, 29, 29, 23,  7, 29,  2,  4,
          4, 29,  3, 29, 20, 29, 29, 29, 22, 29,  8, 29, 29, 29, 29,  6, 29,  8,
          8, 29, 29, 29, 29, 29, 20, 29,  4,  4, 29, 29, 29, 29, 22, 29, 29,  0,
         29, 29, 29, 29, 29, 29, 29, 29, 29, 29,  9, 29, 23,  0, 29, 29, 29, 15,
          4, 29, 29, 29, 29,

In [51]:
# Dekode prediksi ID menjadi teks
transcription = processor.batch_decode(predicted_ids)[0]

print("Transkripsi:", transcription)

Transkripsi: jancok arek iki koen ga ngertia wong jek bingung malah digupui goblok


In [52]:
# Path ke file audio
audio_path = "./audio_testingg2.wav"

In [53]:
# Muat dan preprocess audio
audio_array = load_and_preprocess_audio(audio_path)

# Preprocess audio dengan processor
inputs = processor(audio_array, sampling_rate=16000, return_tensors="pt", padding=True)

# Model inferensi
with torch.no_grad():
    logits = model(inputs.input_values).logits

# Ambil argmax dari logits untuk mendapatkan prediksi ID
predicted_ids = torch.argmax(logits, dim=-1)

In [54]:
# Dekode prediksi ID menjadi teks
transcription = processor.batch_decode(predicted_ids)[0]

print("Transkripsi:", transcription)

Transkripsi: cok a tai lo cok sekteres ku a ngentot men apesi anjing gurus ki maneh gaes gurus gimana anjan burus ki pa


In [55]:
import shutil
import zipfile
import os

# Direktori model yang ingin Anda zip
model_dir = "/content/drive/MyDrive/Skripsi/wav2vec2-profanity-javanese-sby"

# Nama file zip yang akan Anda buat
zip_filename = "model.zip"

# Fungsi untuk membuat file zip
def create_zip_folder(folder_path, zip_path):
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), os.path.join(folder_path, '..')))

# Membuat file zip dari folder model
create_zip_folder(model_dir, zip_filename)

# Men-download file zip
from google.colab import files
files.download(zip_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>